In [ ]:
%matplotlib inline
import sys
sys.path.insert(0, 'src')

In [2]:
import stlcg
import stlviz as viz
from stlcg import Expression
from utils import print_learning_progress

ModuleNotFoundError: No module named 'torch'

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import importlib

## Create traces from numpy arrays

In [ ]:
t = t = np.arange(-3, 3, 0.2, dtype=np.float32)
x_np = 0.5 * np.exp(-t**2).reshape([1, t.shape[0], 1])
w_np = (0.4*np.exp(-(t + 0.5)**2) + 0.2*np.exp(-(t - 3)**2)).reshape([1, t.shape[0], 1])

x = torch.tensor(x_np, requires_grad=False)
w = torch.tensor(w_np, requires_grad=False)
c = torch.tensor(1.0, dtype=torch.float, requires_grad=True)
d = torch.tensor(0.9, dtype=torch.float, requires_grad=True)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(t, x_np[0,:,0], ".-", linewidth=3, markersize=15, label="x")
plt.plot(t, w_np[0,:,0], ".-", linewidth=3, markersize=15, label="w")
plt.legend(fontsize=16)
plt.title("Traces", fontsize=16)

# Constructing STL formulas

### One way to construct formulas is to explicitly call the constructors

In [ ]:
ϕ1 = stlcg.LessThan(lhs='x', val=c)
ϕ2 = stlcg.GreaterThan(lhs='w', val=d)
ϕ3 = stlcg.LessThan(lhs='w', val=d)
ϕ = stlcg.Always(subformula=ϕ1)
ψ = stlcg.Always(subformula=ϕ3)
formula = stlcg.And(subformula1=ϕ, subformula2=ψ)
print(formula)
viz.make_stl_graph(formula)

### Another way (preferred way) is to use the math operators which have been overloaded
#### (less than, greater than, equal, not, and, or)
#### Still need to use the Always and Or constructors explicitly

In [ ]:
x_exp = Expression('x', x)    # The first argument is the name of the variable, the second argument is the numerical value
w_exp = Expression('w', w)
c_exp = Expression('c', c)         # If the variable is a parameter of the STL, then input the actual value
d_exp = Expression('d', d)

In [ ]:
ϕ1 = x_exp <= c_exp
ϕ2 = w_exp >= d_exp 
ϕ3 = w_exp <= d_exp
ϕ = stlcg.Always(subformula=ϕ1)
ψ = stlcg.Always(subformula=ϕ3)
formula = stlcg.And(subformula1=ϕ, subformula2=ψ)
print(formula)
viz.make_stl_graph(formula)

## Evaluating the formula
### You need to feed in the necessary inputs. If the formula requires multiple inputs, it takes in a tuple of length 2 where each element in the tuple is the input to each subformula, in the order they are created. If the formula requires more than two inputs, then the tuple contains a tuple, eg. (x, (y, z)). The grouping of the input traces depend on how the formula was created.

#### the inputs can be made up of tensors, or Expression objects.

In [ ]:
formula2 = ϕ2 & formula
print(formula2)
viz.make_stl_graph(formula2)

In [ ]:
# any of the inputs will work
inputs = (w, (x_exp, w_exp))
# inputs = (w_exp, (x_exp, w_exp))
# inputs = (w, (x, w))

pscale = 1     # "pscale" is the scale used for evaluting predicates
scale = -1     # "scale" is the scale used in the maxish/minish function. <0 defaults to the true min/max
formula2(inputs, pscale=pscale, scale=scale)

# Testing grad functionality

In [ ]:
x_exp = Expression('x', x)
w_exp = Expression('w', w)
c_exp = Expression('c', c)
d_exp = Expression('d', d)
lt = x_exp <= c_exp
gt = w_exp <= d_exp
formula = stlcg.Always(subformula=lt)
inputs = x
var_dict = {'c': c}
print(formula)
viz.make_stl_graph(formula)

In [ ]:
learning_rate = 0.05
device = torch.device("cpu")
optimizer = torch.optim.Adam(var_dict.values(), lr=learning_rate)
scale = 0.5


In [ ]:

for i in range(20000):
    sc = scale + i/500
    loss = formula.robustness(inputs, scale=sc).mean()**2 #+ 0.001*(c**2 + d**2)
    if i % 500 == 0:
        print_learning_progress(formula, inputs, var_dict, i, loss, sc)
    loss.backward()
    with torch.no_grad():
        c -= learning_rate * c.grad
        c.grad.zero_()
#         d -= learning_rate * d.grad
#         d.grad.zero_()

In [ ]:
c